"""
# Análisis de Sentimiento en Reseñas de Juegos

## Introducción
En este proyecto, aplicaremos técnicas de procesamiento de lenguaje natural (NLP) para crear una nueva columna llamada 'sentiment_analysis' en el conjunto de datos 'user_reviews'. Esta columna representará el análisis de sentimiento de las reseñas de juegos realizadas por diferentes usuarios.

## Objetivo
- Asignar valores a la columna 'sentiment_analysis' según la siguiente escala:
    - 0: Reseña mala.
    - 1: Reseña neutral (o ausente).
    - 2: Reseña positiva.
- Reemplazar la columna original 'user_reviews.review' con esta nueva columna para facilitar el trabajo de los modelos de aprendizaje automático y el análisis de datos.

## Pasos a seguir
1. Preprocesamiento de texto: Limpiaremos y normalizaremos las reseñas.
2. Análisis de sentimiento con NLP: Evaluaremos el tono emocional de las reseñas.
3. Reemplazo de columna: La nueva columna 'sentiment_analysis' reemplazará la columna original.

### Lectura:


In [9]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
%load_ext autoreload
%autoreload 2
import utils
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### proceso de extracción de datos

In [2]:
df_reviews = pd.read_csv('steam_reviews_cleaned.csv', encoding='utf-8')
df_games = pd.read_csv('steam_games_cleaned.csv', encoding='utf-8')
df_items = pd.read_csv('data_items_desanidado_cleaned.csv ', encoding='utf-8')

In [3]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   genres     28851 non-null  object 
 1   name       32132 non-null  object 
 2   tags       31971 non-null  object 
 3   specs      31464 non-null  object 
 4   price      28846 non-null  float64
 5   item_id    32133 non-null  float64
 6   developer  28900 non-null  object 
 7   year       32133 non-null  int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 2.0+ MB


# Metodología para el Análisis de Sentimiento con NLTK 

## Introducción
En este proyecto, aplicaremos técnicas de procesamiento de lenguaje natural (NLP) utilizando la biblioteca NLTK en Python. El objetivo es realizar un análisis de sentimiento básico en comentarios de usuarios relacionados con un juego específico. La metodología se centra en asignar valores numéricos a los comentarios para representar si el sentimiento expresado en el texto es negativo, neutral o positivo.


In [4]:
# Reemplazar Nan en valores con espacios
df_reviews.fillna(" ", inplace=True)

In [5]:
# Cargar SentimentIntensityAnalyzer de TLKit
sia = SentimentIntensityAnalyzer()


In [6]:

# crear una instancia con CountVectorizer 
vectorizer = CountVectorizer()

In [7]:

# Convertir los valores en x de review
X = vectorizer.fit_transform(df_reviews["review"])


In [8]:
# Calcular el sentimiento mediante un bucle que itera en cada elemento
sentiments = []
for i in range(X.shape[0]):
    text = df_reviews["review"][i]
    score = sia.polarity_scores(text)["compound"]
    sentiments.append(score)

In [9]:
# agregamos columna sentiment
df_reviews["sentiment"] = sentiments

In [10]:
# mostramos primeros quince valores para verificar 
print(df_reviews["sentiment"].head(15))

0     0.8481
1     0.2263
2     0.9117
3     0.9566
4     0.9708
5     0.7713
6     0.3313
7     0.4767
8     0.0000
9     0.6267
10    0.5106
11    0.8381
12    0.9885
13    0.8922
14    0.7579
Name: sentiment, dtype: float64


In [11]:
# Convierte los valores de sentimiento a una escala de 0 a 2
df_reviews["sentiment"] = df_reviews["sentiment"].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))

In [12]:
porcentaje = df_reviews['sentiment'].value_counts(normalize=True) * 100
print(porcentaje)

sentiment
2    64.199047
1    19.182591
0    16.618362
Name: proportion, dtype: float64


In [13]:
df_tratamientoML = df_reviews[['item_id', 'recommend', 'sentiment']]

In [14]:
df_reviews.head(3)

,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,js41637,http://steamcommunity.com/id/js41637,1,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.,2
2,evcentric,http://steamcommunity.com/id/evcentric,2,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2


In [15]:
df_tratamientoML

,item_id,recommend,sentiment
0,1250,True,2
1,22200,True,2
2,43110,True,2
3,251610,True,2
4,227300,True,2
...,...,...,...
20352,55230,True,2
20353,440,True,2
20354,215280,True,2
20355,8500,False,0


In [16]:
df_tratamientoML = df_tratamientoML.merge(df_games[['item_id', 'year']], on='item_id', how='left')
df_tratamientoML

,item_id,recommend,sentiment,year
0,1250,True,2,2009.0
1,22200,True,2,2009.0
2,43110,True,2,NaN
3,251610,True,2,NaN
4,227300,True,2,2013.0
...,...,...,...,...
20352,55230,True,2,2011.0
20353,440,True,2,2007.0
20354,215280,True,2,2017.0
20355,8500,False,0,2003.0


In [18]:
# vamos a tratar los valores Nan con el metodo interpolate para completar valores faltantes.
df_tratamientoML['year'].interpolate(method='linear', inplace=True)

In [59]:
# Los archivos se almacenan en local 
df_tratamientoML.to_csv('user_review_tratamientoML.csv', index=False)

## Función desarrollador: 

La función desarrollador toma un argumento desarrollador que es una cadena que representa el nombre de la empresa desarrolladora. La función extrae los datos relevantes de la empresa desarrolladora y devuelve la cantidad de elementos y el porcentaje de contenido gratuito por año.

In [20]:
# Se extraen las columnas 'items_count', 'user_id' e 'item_id'
df_gastos_items = df_items[['items_count', 'user_id', 'item_id']]
df_gastos_items

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50
...,...,...,...
5094100,7,76561198329548331,346330
5094101,7,76561198329548331,373330
5094102,7,76561198329548331,388490
5094103,7,76561198329548331,521570


In [21]:
price_juegos = df_games[['price', 'item_id']]
# Elimina los duplicados
price_juegos = price_juegos.drop_duplicates(subset='item_id', keep='first')
# Se renombra 'id' por 'item_id' para unir mas adelante
price_juegos = price_juegos.rename(columns={'id':'item_id'})


In [22]:
price_juegos['price'].fillna(0, inplace=True)

In [23]:
df_gastos_items = df_gastos_items.merge(price_juegos, on='item_id', how='left')


In [24]:
df_gastos_items['price'].fillna(0, inplace=True)
df_gastos_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094100,7,76561198329548331,346330,0.00
5094101,7,76561198329548331,373330,0.00
5094102,7,76561198329548331,388490,0.00
5094103,7,76561198329548331,521570,0.00


In [25]:
price_juegos

,price,item_id
0,4.99,761140.0
1,0.00,643980.0
2,0.00,670290.0
3,0.99,767400.0
4,2.99,773570.0
...,...,...
32128,1.99,773640.0
32129,4.99,733530.0
32130,1.99,610660.0
32131,4.99,658870.0


In [26]:
# Calcular el porcentaje de valores no nulos y nulos para cada columna
no_nulos_por_columna = df_gastos_items.count() / len(df_gastos_items) * 100
nulos_por_columna = df_gastos_items.isnull().sum() / len(df_gastos_items) * 100

# Crear un DataFrame con los resultados
resultados = pd.concat([df_gastos_items.dtypes, no_nulos_por_columna, nulos_por_columna, df_gastos_items.isnull().sum()], axis=1)
resultados.columns = ['tipo_datos', 'no_nulos_%', 'nulos_%', 'nulos']
resultados.index.name = 'nombre_campo'

# Mostrar los resultados
print(resultados.to_string())

             tipo_datos  no_nulos_%  nulos_%  nulos
nombre_campo                                       
items_count       int64       100.0      0.0      0
user_id          object       100.0      0.0      0
item_id           int64       100.0      0.0      0
price           float64       100.0      0.0      0


In [27]:
df_gastos_items['price']

0          9.99
1          4.99
2          4.99
3          4.99
4          4.99
           ... 
5094100    0.00
5094101    0.00
5094102    0.00
5094103    0.00
5094104    0.00
Name: price, Length: 5094105, dtype: float64

In [28]:
df_gastos_items['price'].value_counts(dropna=False)

price
0.00      1733370
9.99       856043
19.99      794457
14.99      471881
4.99       327745
           ...   
124.99          2
1.50            1
119.99          1
74.99           1
2.89            1
Name: count, Length: 87, dtype: int64

Como podemos observar no se encuentran valores nulos en la columna price por tanto podemos contar con esta columna para comenzar a definir la función sin embargo debemos exporar  developer  y year. 

In [29]:
# Se elimina 'item_id'
df_gastos_items = df_gastos_items.drop('item_id', axis=1)
df_gastos_items.columns

Index(['items_count', 'user_id', 'price'], dtype='object')

In [30]:
# Se agrupa por usuario sumando los precios de los juegos consumidos
df_gastos_items_group = df_gastos_items.groupby('user_id')['price'].sum().reset_index()
df_gastos_items_group

,user_id,price
0,--000--,402.77
1,--ace--,166.82
2,--ionex--,109.92
3,-2SV-vuLB-Kg,437.49
4,-404PageNotFound-,1514.31
...,...,...
70907,zzonci,19.98
70908,zzoptimuszz,64.98
70909,zzydrax,99.94
70910,zzyfo,828.51


Para volver a agregar los items consumidos por cada usuario se hace un dataframe auxiliar 'df_count_items' conteniendo la cantidad de items por usuario.

In [31]:
df_count_items = df_gastos_items[['items_count', 'user_id']]
# se eliminan los duplicados
df_count_items = df_count_items.drop_duplicates(subset='user_id', keep='first')
df_count_items

,items_count,user_id
0,277,76561197970982479
277,888,js41637
1165,137,evcentric
1302,328,Riot-Punch
1630,541,doctr
...,...,...
5093574,321,76561198320136420
5093895,4,ArkPlays7
5093899,22,76561198323066619
5093921,177,76561198326700687


Finalmente, se juntan los dos dataframe auxiliares 'df_gastos_items_group' que contiene el gasto de cada usuario con 'df_count_items' que contiene la cantidad de items consumidos por cada usuario.

In [32]:
df_gastos_items = df_count_items.merge(df_gastos_items_group, on='user_id', how='right')
df_gastos_items

,items_count,user_id,price
0,58,--000--,402.77
1,44,--ace--,166.82
2,23,--ionex--,109.92
3,68,-2SV-vuLB-Kg,437.49
4,149,-404PageNotFound-,1514.31
...,...,...,...
70907,5,zzonci,19.98
70908,61,zzoptimuszz,64.98
70909,13,zzydrax,99.94
70910,84,zzyfo,828.51


## Playtime_forever por usuario

Se busca generar un dataframe que contenga por género de videojuego, el tiempo jugado por cada usuario con su id y url del perfil. Para ello, se realizarán algunos pasos generando algunos dataframes auxiliares.
En primer lugar, se extrae de df_items las columnas 'playtime_forever', 'user_id' y 'item_id' y se reserva.

In [33]:
playtime_forever_usuario_item = df_items[['playtime_forever', 'user_id', 'item_id']]
playtime_forever_usuario_item

,playtime_forever,user_id,item_id
0,6,76561197970982479,10
1,0,76561197970982479,20
2,7,76561197970982479,30
3,0,76561197970982479,40
4,0,76561197970982479,50
...,...,...,...
5094100,0,76561198329548331,346330
5094101,0,76561198329548331,373330
5094102,3,76561198329548331,388490
5094103,4,76561198329548331,521570


In [34]:
genre_item = df_games[['genres', 'item_id']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
genre_item = genre_item.rename(columns={'id':'item_id'})
genre_item

,genres,item_id
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",761140.0
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",643980.0
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",670290.0
3,"['Action', 'Adventure', 'Casual']",767400.0
4,NaN,773570.0
...,...,...
32128,"['Casual', 'Indie', 'Simulation', 'Strategy']",773640.0
32129,"['Casual', 'Indie', 'Strategy']",733530.0
32130,"['Indie', 'Racing', 'Simulation']",610660.0
32131,"['Casual', 'Indie']",658870.0


In [35]:
df_playtime_genre = playtime_forever_usuario_item.merge(genre_item, on='item_id')
df_playtime_genre

,playtime_forever,user_id,item_id,genres
0,6,76561197970982479,10,['Action']
1,0,76561197970982479,20,['Action']
2,7,76561197970982479,30,['Action']
3,0,76561197970982479,40,['Action']
4,0,76561197970982479,50,['Action']
...,...,...,...,...
4244826,43,76561198329548331,227940,"['Action', 'Free to Play', 'Indie', 'Massively..."
4244827,0,76561198329548331,346330,"['Action', 'Adventure', 'Free to Play', 'Indie..."
4244828,0,76561198329548331,373330,NaN
4244829,3,76561198329548331,388490,"['Adventure', 'Free to Play']"


Se agrupa por género y usuario y se suma el tiempo de juego de para cada caso, se asume que la columna playtime_forever se encuentra en minutos, por lo que se transforma a horas.

In [36]:
# Se agrupa por usuario y se suma el tiempo de juego
agg_genero = df_playtime_genre.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()
# Se transforma la columna 'playtime_forever' a horas
agg_genero['playtime_horas'] = agg_genero['playtime_forever']/60
# Se borra la columna en minutos
agg_genero = agg_genero.drop('playtime_forever', axis=1)
agg_genero

,genres,user_id,playtime_horas
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333
1,"['Action', 'Adventure', 'Casual', 'Free to Pla...",666999661,15.550000
2,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197967469174,0.000000
3,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197983367658,0.000000
4,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197985797191,0.000000
...,...,...,...
1704211,['Web Publishing'],xjerseypsycho,0.000000
1704212,['Web Publishing'],xxmaster01,0.000000
1704213,['Web Publishing'],yuvii28,0.000000
1704214,['Web Publishing'],zachster,0.583333


Se quiere tener el url del perfil del usuario, por lo que se hace un dataframe auxiliar del 'user_id' con 'user_url', se eliminan los duplicados para que queden usuarios sin repetir y finalmente se agrega al dataframe auxiliar anterior para generar finalmente el dataframe 'df_playtime_forever' que contiene por género las horas de juego por usuario y su url.

In [37]:
user_url_usuario = df_items[['user_url', 'user_id']]
# Se eliminan nuplicados para tener una sola vez los 'user_id' con su url
user_url_usuario= user_url_usuario.drop_duplicates(subset='user_id', keep='first')
# Se une con el dataframe agregado de género
df_playtime_forever = agg_genero.merge(user_url_usuario, on='user_id', how='left')
df_playtime_forever.head()

,genres,user_id,playtime_horas,user_url
0,"['Action', 'Adventure', 'Casual', 'Free to Pla...",12das,0.133333,http://steamcommunity.com/id/12das
1,"['Action', 'Adventure', 'Casual', 'Free to Pla...",666999661,15.550000,http://steamcommunity.com/id/666999661
2,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197967469174,0.000000,http://steamcommunity.com/profiles/76561197967...
3,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197983367658,0.000000,http://steamcommunity.com/profiles/76561197983...
4,"['Action', 'Adventure', 'Casual', 'Free to Pla...",76561197985797191,0.000000,http://steamcommunity.com/profiles/76561197985...


## Ranking de géneros por tiempo de juego

En este punto lo que se busca es un dataframe auxiliar que contenga el ranking de los géneros de videojuegos en función del tiempo jugados de cada uno. Se aprovecha que ya se hizo para el punto anterior un dataframe auxiliar que contiene los géneros y el tiempo jugado, por lo que se usa el mismo. Primero se agrupa por género, luego se ordena y se agrega una columna de ranking.

In [38]:
df_genre_ranking = agg_genero.groupby('genres')['playtime_horas'].sum().reset_index()
# Se ordena por 'playtime_horas'
df_genre_ranking = df_genre_ranking.sort_values(by='playtime_horas', ascending=False)
# Agregar una columna con la posición del ranking
df_genre_ranking['ranking'] = df_genre_ranking['playtime_horas'].rank(ascending=False).astype(int)
df_genre_ranking

,genres,playtime_horas,ranking
262,['Action'],2.205491e+07,1
528,"['Indie', 'Simulation']",7.786394e+06,2
112,"['Action', 'Adventure']",4.129597e+06,3
243,"['Action', 'RPG']",3.973749e+06,4
583,['Strategy'],3.944872e+06,5
...,...,...,...
385,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",0.000000e+00,582
441,"['Design &amp; Illustration', 'Education']",0.000000e+00,582
540,"['Photo Editing', 'Software Training', 'Utilit...",0.000000e+00,582
286,"['Adventure', 'Casual', 'Indie', 'Sports']",0.000000e+00,582


## Cantidad de items y contenido free por desarrollador

En este punto se busca un dataframe que contenga para cada desarrollador de juegos, los items que desarrolla, el año de lanzamiento y el precio de cada uno.

In [39]:
price_anio_developer = df_games[['price', 'year', 'developer', 'item_id']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
price_anio_developer = price_anio_developer.rename(columns={'id':'item_id'})
# se eliminan los duplicados
df_items_developer = price_anio_developer.drop_duplicates()
df_items_developer

,price,year,developer,item_id
0,4.99,2018,Kotoshiro,761140.0
1,NaN,2018,Secret Level SRL,643980.0
2,NaN,2017,Poolians.com,670290.0
3,0.99,2017,彼岸领域,767400.0
4,2.99,2016,NaN,773570.0
...,...,...,...,...
32128,1.99,2018,"Nikita ""Ghost_RUS""",773640.0
32129,4.99,2018,Sacada,733530.0
32130,1.99,2018,Laush Dmitriy Sergeevich,610660.0
32131,4.99,2017,"xropi,stev3ns",658870.0


In [40]:
df_items_developer[['price', 'year', 'developer', 'item_id']] = df_items_developer[['price', 'year', 'developer', 'item_id']].fillna(0)

In [41]:
df_items_developer

,price,year,developer,item_id
0,4.99,2018,Kotoshiro,761140.0
1,0.00,2018,Secret Level SRL,643980.0
2,0.00,2017,Poolians.com,670290.0
3,0.99,2017,彼岸领域,767400.0
4,2.99,2016,0,773570.0
...,...,...,...,...
32128,1.99,2018,"Nikita ""Ghost_RUS""",773640.0
32129,4.99,2018,Sacada,733530.0
32130,1.99,2018,Laush Dmitriy Sergeevich,610660.0
32131,4.99,2017,"xropi,stev3ns",658870.0


In [42]:
df_items_developer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32132
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price      32132 non-null  float64
 1   year       32132 non-null  int64  
 2   developer  32132 non-null  object 
 3   item_id    32132 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.2+ MB


In [43]:
# Agrupar por año y contar la cantidad de desarrolladores con contenido gratuito
desarrolladores_por_año = df_items_developer[df_items_developer['price'] == 0].groupby('year')['developer'].nunique()

# Crear un DataFrame con los resultados
resultados = pd.DataFrame({
    'Año': desarrolladores_por_año.index,
    'Desarrolladores con contenido gratuito': desarrolladores_por_año.values
})

# Calcular el porcentaje de desarrolladores con contenido gratuito
total_desarrolladores = df_items_developer['developer'].nunique()
resultados['Porcentaje Desarrolladores Gratis'] = (resultados['Desarrolladores con contenido gratuito'] / total_desarrolladores) * 100

# Mostrar los resultados
print(resultados)

     Año  Desarrolladores con contenido gratuito  \
0   1981                                       1   
1   1990                                       1   
2   1991                                       2   
3   1993                                       1   
4   1994                                       2   
5   1995                                       1   
6   1996                                       2   
7   1997                                       1   
8   1998                                       1   
9   1999                                       4   
10  2001                                       3   
11  2002                                       1   
12  2003                                       3   
13  2004                                       2   
14  2005                                       6   
15  2006                                       7   
16  2007                                      16   
17  2008                                      17   
18  2009    

In [44]:
df_items_developer['developer'].head(20)

0                          Kotoshiro
1                   Secret Level SRL
2                       Poolians.com
3                               彼岸领域
4                                  0
5                Trickjump Games Ltd
6             Poppermost Productions
7             Poppermost Productions
8             Poppermost Productions
9                          RewindApp
10                           Qucheza
11                                 0
12    Stegalosaurus Game Development
13                 Copperpick Studio
14                      Ghulam Jewel
15                            Apillo
16            Poppermost Productions
17                            Apillo
18            Poppermost Productions
19                                 0
Name: developer, dtype: object

In [45]:
df_items_developer['developer'] = df_items_developer['developer'].astype(str)


In [46]:
# Agrupar los datos por año y desarrollador y contar el número de elementos gratuitos y de pago
df_grouped = df_items_developer.groupby(['year', 'developer'])['price'].value_counts().reset_index(name='count')

# Crear un nuevo dataframe con los resultados
df_results = pd.DataFrame({'año': df_grouped['year'], 'Desarrollador': df_grouped['developer'], 'Tipo de contenido': df_grouped['price']})

# Agregar una columna que indique si el contenido es gratuito o de pago
df_results['Tipo de contenido'] = df_results['Tipo de contenido'].apply(lambda x: 'Contenido gratuito' if x == 0 else 'Contenido de pago')

# Imprimir los resultados
print(df_results)

        año           Desarrollador   Tipo de contenido
0      1970                       0   Contenido de pago
1      1975                       0   Contenido de pago
2      1980                       0   Contenido de pago
3      1981                       0   Contenido de pago
4      1981                       0  Contenido gratuito
...     ...                     ...                 ...
20795  2018            杭州分浪网络科技有限公司  Contenido gratuito
20796  2019       Ninetales Studios   Contenido de pago
20797  2019     Nodding Heads Games  Contenido gratuito
20798  2019  Poetic Justice Studios  Contenido gratuito
20799  2021          Freebird Games   Contenido de pago

[20800 rows x 3 columns]


In [47]:
df_results

,año,Desarrollador,Tipo de contenido
0,1970,0,Contenido de pago
1,1975,0,Contenido de pago
2,1980,0,Contenido de pago
3,1981,0,Contenido de pago
4,1981,0,Contenido gratuito
...,...,...,...
20795,2018,杭州分浪网络科技有限公司,Contenido gratuito
20796,2019,Ninetales Studios,Contenido de pago
20797,2019,Nodding Heads Games,Contenido gratuito
20798,2019,Poetic Justice Studios,Contenido gratuito


relacionamos los datos para poder hacer posible la función desarrollador 

In [34]:
# Los archivos se almacenan en local 
df_items_developer.to_csv('funcion_desarrollador.csv', index=False)


In [13]:
df_items_developer.to_parquet('funcion_desarrollador', index=False)

In [34]:
df_items_developer

,price,year,developer,item_id
0,4.99,2018,Kotoshiro,761140.0
1,NaN,2018,Secret Level SRL,643980.0
2,NaN,2017,Poolians.com,670290.0
3,0.99,2017,彼岸领域,767400.0
4,2.99,2016,NaN,773570.0
...,...,...,...,...
32128,1.99,2018,"Nikita ""Ghost_RUS""",773640.0
32129,4.99,2018,Sacada,733530.0
32130,1.99,2018,Laush Dmitriy Sergeevich,610660.0
32131,4.99,2017,"xropi,stev3ns",658870.0


In [8]:
def developer(desarrollador):
    df = pd.read_parquet('funcion_desarrollador')
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df[df['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('year')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('year')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)

    result_dict = {
        'cantidad_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict

In [9]:
developer("RewindApp")


{'cantidad_por_año': {2017: 8, 2018: 1},
 'porcentaje_gratis_por_año': {2017: 0, 2018: 100}}

In [49]:
df_reviews.to_parquet('df_reviews.parquet')
df_games.to_parquet('df_games.parquet')
df_items.to_parquet('df_items.parquet')
df_gastos_items.to_parquet('df_gastos_items.parquet')
df_genre_ranking.to_parquet('df_genre_ranking.parquet')
df_playtime_forever.to_parquet('df_playtime_forever.parquet')
df_items_developer.to_parquet('df_items_developer.parquet')



In [58]:
df_reviews = pd.read_parquet('df_reviews.parquet')
df_gastos_items = pd.read_parquet('df_gastos_items.parquet')   
df_genre_ranking = pd.read_parquet('df_genre_ranking.parquet')
df_playtime_forever = pd.read_parquet('df_playtime_forever.parquet')   


In [65]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20357 entries, 0 to 20356
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      20357 non-null  object
 1   user_url     20357 non-null  object
 2   index        20357 non-null  int64 
 3   funny        20357 non-null  object
 4   posted       20357 non-null  object
 5   last_edited  20357 non-null  object
 6   item_id      20357 non-null  int64 
 7   helpful      20357 non-null  object
 8   recommend    20357 non-null  bool  
 9   review       20357 non-null  object
 10  sentiment    20357 non-null  int64 
dtypes: bool(1), int64(3), object(7)
memory usage: 1.6+ MB


In [66]:
# Crear la columna "reviews_recommend"
df_reviews['reviews_recommend'] = df_reviews['recommend'].apply(lambda x: 'Recomendado' if x else 'No recomendado')

# Mostrar las primeras filas del DataFrame actualizado
print(df_reviews.head())

             user_id                                           user_url  \
0  76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1            js41637               http://steamcommunity.com/id/js41637   
2          evcentric             http://steamcommunity.com/id/evcentric   
3              doctr                 http://steamcommunity.com/id/doctr   
4          maplemage             http://steamcommunity.com/id/maplemage   

   index funny      posted last_edited  item_id  \
0      0        2011-11-05                 1250   
1      1        2011-07-15                22200   
2      2        2011-04-21                43110   
3      3        2014-06-24               251610   
4      4        2013-09-08               227300   

                                           helpful  recommend  \
0                                   No ratings yet       True   
1                                   No ratings yet       True   
2                                   No ratings 

In [75]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20357 entries, 0 to 20356
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            20357 non-null  object
 1   user_url           20357 non-null  object
 2   index              20357 non-null  int64 
 3   funny              20357 non-null  object
 4   posted             20357 non-null  object
 5   last_edited        20357 non-null  object
 6   item_id            20357 non-null  int64 
 7   helpful            20357 non-null  object
 8   recommend          20357 non-null  bool  
 9   review             20357 non-null  object
 10  sentiment          20357 non-null  int64 
 11  reviews_recommend  20357 non-null  object
dtypes: bool(1), int64(3), object(8)
memory usage: 1.7+ MB


In [82]:
def userdata(user_id):
  
      try:
        # Filtra por el usuario de interés
        usuario = df_reviews[df_reviews['user_id'] == user_id]
        
        # Calcula la cantidad de dinero gastado para el usuario de interés
        cantidad_dinero = float(df_gastos_items[df_gastos_items['user_id'] == user_id]['price'].iloc[0])
        
        # Busca el count_items para el usuario de interés
        count_items = int(df_gastos_items[df_gastos_items['user_id'] == user_id]['items_count'].iloc[0])
        
        # Filtra las recomendaciones del usuario de interés
        recomendaciones_usuario = usuario['reviews_recommend']
        
        # Calcula el total de recomendaciones realizadas por el usuario de interés
        total_recomendaciones = recomendaciones_usuario[recomendaciones_usuario == 'Recomendado'].count()
        
        # Calcula el total de reviews realizada por todos los usuarios
        total_reviews = len(df_reviews['user_id'].unique())
        
        # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
        porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
        
        return {
            'cantidad_dinero': cantidad_dinero,
            'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
            'total_items': count_items
        }
    except KeyError:
        return {
            'error': 'Usuario no encontrado en los DataFrames'
        }

In [83]:
userdata('js41637')

{'cantidad_dinero': 8553.07,
 'porcentaje_recomendacion': 0.0,
 'total_items': 888}

In [85]:
df_reviews.to_parquet('df_reviews_1.parquet')

In [10]:
def genre(genero):

    df_genre_ranking = pd.read_parquet('df_genre_ranking.parquet')
    
    try:
        # Busca el ranking para el género de interés
        rank = df_genre_ranking.loc[df_genre_ranking['genres'] == genero, 'ranking'].iloc[0]
    except IndexError:
        # Manejo del error: género no encontrado
        rank = None
    return {
        'rank': rank
    }

def userforgenre(genero):
    '''
    Esta función devuelve el top 5 de usuarios con más horas de juego en un género específico, junto con su URL de perfil y ID de usuario.
   l' (str): URL del perfil del usuario.
    '''
    df_playtime_forever = pd.read_parquet('df_playtime_forever.parquet')
    
    # Filtra el dataframe por el género de interés
    data_por_genero = df_playtime_forever[df_playtime_forever['genres'].str.contains(genero)]

    # Agrupa el dataframe filtrado por usuario y suma la cantidad de horas
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()

    # Crear una lista de diccionarios con la información de los usuarios
    top_users_list = []
    for index, row in top_users.iterrows():
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users_list.append(user_info)

    # Crear el diccionario de retorno
    result = {
        'Top 5 usuarios para el género ' + genero: top_users_list
    }
    return result

In [11]:
userforgenre('Action')

{'Top 5 usuarios para el género Action': [{'user_id': 'Sp3ctre',
   'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
  {'user_id': 'shinomegami',
   'user_url': 'http://steamcommunity.com/id/shinomegami'},
  {'user_id': 'REBAS_AS_F-T',
   'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
  {'user_id': 'Terminally-Chill',
   'user_url': 'http://steamcommunity.com/id/Terminally-Chill'},
  {'user_id': 'DownSyndromeKid',
   'user_url': 'http://steamcommunity.com/id/DownSyndromeKid'}]}

In [16]:
def obtener_top_desarrolladores(df_playtime_forever, year):

    df_playtime_forever = pd.read_parquet('df_playtime_forever.parquet')

    # Filtrar las reseñas con comentarios positivos y recomendaciones verdaderas
    filtered_df = df_playtime_forever[df_playtime_forever['playtime_horas'] > 0]
    
    # Agrupar por desarrollador y calcular la suma de las recomendaciones
    developer_scores = filtered_df.groupby('user_id')['playtime_horas'].sum()
    
    # Ordenar los desarrolladores por puntuación en orden descendente
    sorted_developers = developer_scores.sort_values(ascending=False)
    
    # Seleccionar los tres primeros desarrolladores
    top_3_developers = sorted_developers.head(3)
    
    # Crear una lista de diccionarios con el formato deseado
    result = [{"Puesto {}: {}".format(i + 1, dev): score} for i, (dev, score) in enumerate(top_3_developers.items())]
    
    return result

def best_developer_year(year):
    df_playtime_forever = pd.read_parquet('df_playtime_forever.parquet')
    return obtener_top_desarrolladores(df_playtime_forever, year)

In [17]:
year = 2024
top_developers = best_developer_year(year)
print(top_developers)


[{'Puesto 1: REBAS_AS_F-T': 65640.35}, {'Puesto 2: shinomegami': 50957.36666666667}, {'Puesto 3: DownSyndromeKid': 40474.083333333336}]
